In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [51]:
url_inicial='https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/archive/3'
url_root= 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/archive/3'
r = requests.get(url_inicial)

In [52]:
soup = BeautifulSoup(r.text, 'html.parser')

In [53]:
box = soup.find('div', class_='page page_issue_archive')
volumen=box.findAll('div', class_='card obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/39',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/40',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/41',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/42',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/43',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/44',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/45',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/46',
 'https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/view/47']

In [54]:
def get_url_items(sopa, url): 
    box = soup.find('div', class_='page page_issue_archive')
    volumen=box.findAll('div', class_='card obj_issue_summary')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [55]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('div.cmp_pagination > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Estoy en la pagina https://perfilesla.flacso.edu.mx/index.php/perfilesla/issue/archive/3


In [56]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

9

In [57]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [58]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Perfiles Latinoamericanos'
    a='V. Ciencias Sociales'
    tem='Ciencia Política; Sociología; Economía, Políticas Públicas; Educación; Población y Desarrollo'
    url='http://perfilesla.flacso.edu.mx/index.php/perfilesla/index'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('li', class_='breadcrumb-item active').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', class_='journal-logo div_imagen_banner').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('div', class_='card one-article-intoc').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [59]:
list_scraper=list_scraper[0:9]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8


In [60]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 5 Núm. 9 (1996),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
1,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 5 Núm. 8 (1996),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
2,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 4 Núm. 7 (1995),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
3,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 4 Núm. 6 (1995),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
4,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 3 Núm. 5 (1994),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
5,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 3 Núm. 4 (1994),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
6,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 2 Núm. 3 (1993),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
7,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 2 Núm. 2 (1993),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...
8,Perfiles Latinoamericanos,V. Ciencias Sociales,"Ciencia Política; Sociología; Economía, Políti...",Vol. 1 Núm. 1 (1992),None,http://perfilesla.flacso.edu.mx/index.php/perf...,https://perfilesla.flacso.edu.mx/index.php/per...


In [61]:
df_catalogo.to_csv('Revista21.3.csv', index=False)